In [1]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [12]:
df = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

In [25]:
resumen_egresos_nacionales = (
    df.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

In [49]:
diags_ranking_1_o_2_torax = resumen_egresos_nacionales.filter(
    (pl.col("COD_HOSPITAL") == 112103)
    & ((pl.col("ranking_egresos")) == 1 | (pl.col("ranking_egresos") == 2))
).sort(["ANIO_EGRESO", "n_egresos"], descending=True)

In [50]:
diags_ranking_1_o_2_torax

ANIO_EGRESO,COD_HOSPITAL,DIAGNOSTICO1,n_egresos,dias_estada_totales,ranking_egresos
i64,i64,str,u32,i64,u32
2021,112103,"""I35.0""",122,1404,1
2021,112103,"""Z45.0""",109,124,1
2021,112103,"""C34.1""",74,550,1
2021,112103,"""J95.5""",68,718,1
2021,112103,"""C78.2""",65,754,1
2021,112103,"""J84.1""",63,998,1
2021,112103,"""C34.3""",61,480,1
2021,112103,"""E84.8""",51,1121,1
2021,112103,"""Q21.1""",50,254,1
